In [3]:
# Billing & Revenue Analytics

## Objective
#Analyze hospital billing data to understand revenue performance, payment status distribution,
#payment method effectiveness, and revenue trends over time.


In [4]:
import pandas as pd
import os


In [5]:
BASE_PATH = "C:/Users/ra481/OneDrive/Desktop/healthcare-analytics"
DATA_PATH = f"{BASE_PATH}/Data/Raw"
REPORTS_PATH = f"{BASE_PATH}/reports"

os.makedirs(REPORTS_PATH, exist_ok=True)


In [6]:
billing_df = pd.read_csv(f"{DATA_PATH}/billing.csv")
billing_df.head()


,bill_id,patient_id,treatment_id,bill_date,amount,payment_method,payment_status
0,B001,P034,T001,2023-08-09,3941.97,Insurance,Pending
1,B002,P032,T002,2023-06-09,4158.44,Insurance,Paid
2,B003,P048,T003,2023-06-28,3731.55,Insurance,Paid
3,B004,P025,T004,2023-09-01,4799.86,Insurance,Failed
4,B005,P040,T005,2023-07-06,582.05,Credit Card,Pending


In [7]:
billing_df.shape
billing_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   bill_id         200 non-null    object 
 1   patient_id      200 non-null    object 
 2   treatment_id    200 non-null    object 
 3   bill_date       200 non-null    object 
 4   amount          200 non-null    float64
 5   payment_method  200 non-null    object 
 6   payment_status  200 non-null    object 
dtypes: float64(1), object(6)
memory usage: 11.1+ KB


In [10]:
billing_df["bill_date"] = pd.to_datetime(billing_df["bill_date"])


In [12]:
billing_df.dtypes


bill_id                   object
patient_id                object
treatment_id              object
bill_date         datetime64[ns]
amount                   float64
payment_method            object
payment_status            object
dtype: object

In [13]:
total_revenue = billing_df["amount"].sum()
total_revenue


np.float64(551249.8500000001)

In [14]:
revenue_by_status = (
    billing_df
    .groupby("payment_status")["amount"]
    .sum()
    .reset_index(name="total_amount")
)

revenue_by_status


,payment_status,total_amount
0,Failed,193212.94
1,Paid,173424.90
2,Pending,184612.01


In [15]:
payment_status_pct = (
    billing_df["payment_status"]
    .value_counts(normalize=True)
    .mul(100)
    .round(2)
    .reset_index()
)

payment_status_pct.columns = ["payment_status", "percentage"]
payment_status_pct


,payment_status,percentage
0,Pending,34.5
1,Failed,33.5
2,Paid,32.0


In [16]:
revenue_by_method = (
    billing_df
    .groupby("payment_method")["amount"]
    .sum()
    .reset_index(name="total_revenue")
    .sort_values(by="total_revenue", ascending=False)
)

revenue_by_method


,payment_method,total_revenue
1,Credit Card,201382.43
2,Insurance,182160.28
0,Cash,167707.14


In [17]:
monthly_revenue = (
    billing_df
    .groupby(billing_df["bill_date"].dt.to_period("M"))["amount"]
    .sum()
    .reset_index(name="monthly_revenue")
)

monthly_revenue["bill_date"] = monthly_revenue["bill_date"].astype(str)
monthly_revenue


,bill_date,monthly_revenue
0,2023-01,58701.23
1,2023-02,36669.69
2,2023-03,47304.29
3,2023-04,64271.54
4,2023-05,48791.05
5,2023-06,56887.82
6,2023-07,39880.19
7,2023-08,41958.67
8,2023-09,33426.53
9,2023-10,43314.15


In [18]:
reports_path = "C:/Users/ra481/OneDrive/Desktop/healthcare-analytics/reports"

revenue_by_status.to_csv(f"{reports_path}/revenue_by_status.csv", index=False)
payment_status_pct.to_csv(f"{reports_path}/payment_status_distribution.csv", index=False)
revenue_by_method.to_csv(f"{reports_path}/revenue_by_method.csv", index=False)
monthly_revenue.to_csv(f"{reports_path}/monthly_revenue.csv", index=False)


In [19]:
import os
os.path.exists(f"{reports_path}/monthly_revenue.csv")


True